# ACSE-4 Project 2: A Smoothed Particle Hydrodynamics Solver

## 1. Description

### a. Algorithm

Our software uses this general class structure. The SPH implementation is in `SPH_stub.py`, this file uses `particle.py` to generate the particle objects. The `simulate` function is the main function that runs in `SPH_stub.py`. It updates the particles in the `particle_list` for each time-step and writes the updated particles object in a pickle file. 

The pickle file can then be used as an input to the `animation.py` script and a mp4 video showing the particles moving in time is generated. 

Our simulation can either be run using the command line interface (with arguments; more details are given in the README file) or for the more savvy python users, we provide scripts that can be edited with the parameters i.e. 

On the command-line, run:

`python run_simulation.py` 

This calls the `simulate` function and creates a pickle file called `datafile.pkl`. 

Then to create the video using this file generated, simpy run:

`python animation.py`


<img src="figures/class_chart.png" style="width: 520px">
<br><br>
The solution algorithm is written as the method *simulate* in *SPH_main*, uses many functions and returns the calculated output as a datafile, according to the diagram below.
<img src="figures/overview_of_simulate_function.png" style="width: 400px">


You will find below the corresponding pseudocode of the method:
<br>
Method simulate(self)
<br>    t = 0
<br>    counter = 0
<br>While t < t_max:
<br>    counter = counter + 1
<br>    smooth = False
<br>    If counter % 10 == 0:
<br>        smooth density = True
<br>    End if
<br>    do scheme and smooth density
<br>    t = t + dt
<br>End while
<br>End Method


This method modifies at each time step the attributes of the particles from *particle_list* using a scheme (forward Euler or predictor corrector). These schemes use Navier-Stokes equations (getting acceleration and time derivative of density) to update all the values (position x, velocity v and density \rho).
It returns ... which is then treated in order to do the animation.


SPH_main:
=> main simulation loops + simulation steps required within these loops

*simulate*:
simulate (while loop) => forward euler or predictor corrector (one time step) => navier stokes (calculating acceleration and time derivative of density) => saving at each time step in a file the current time and the particle with its attributes updated.


then: production of data file + animation (video .mp4)

## 2.  Optimisation

### a. Optimising the computational time

To make the computational time quicker, we implemented a **stencil** to half the time taken to get the neighbours of each of the particles. 

The diagram below shows us a picture of how the stencil looked and which buckets we will be calculating the neighbours from. 

<img src="figures/stencil.png" style="width: 300px">

** Video of FE with stencil and without stencil **

Therefore, we can see that our stencil nearly halves the computational time, which was the goal. This is extremely important especially when running on a finer mesh grid (e.g. $\Delta x = 0.2$), since python takes a *very* long time, finding a way to optimise is necessary.

### b. Improving accuracy

The forward-euler method is only first-order method. However, it is necessary for us to have a more accurate method in time as so we implemented the predictor-corrector method. This method is second-order accurate and uses 2 steps to get a full step and is part of the Runga-Kutta family.

**Implementation**
The pseudo-code: 

**TODO pseudo code**

We can look at the predictor-corrector method as a 2-step forward euler time step. So, in our code, we used our forward-euler function and ran it twice in a for-loop. We have defined different equations (from half and full step of P-C method) and these equations change according to n in the for-loop.   Also, we added extra attributes to each particle in order to store the velocity and position at the previous half-step. 

** Difference between FE and PC video**
** Parameters used in the videos**

#### Comparison between FE and PC schemes

##### Simulation using FE scheme

<video controls src="figures/FE.mp4" />

##### Simulation using PC scheme

<video controls src="figures/Predictor-corrector.mp4" />

These videos show us the difference in behaviour of the particles using the different schemes. The Predictor-Corrector method seems to be more realistic, but runs slower. This is expected as P-C takes 2 half-steps while FE only takes one, so although the accuracy has increased, we lose some computational time. 

# 3. Issues faced

We faced an issue regarding the particles leaking out of the system, as seen in the following video.
<video controls src="figures/leak.mp4" />



We enforced a repulsive force on the wall to stop this from happening. The pseudo-code we based our implementation on is written below:

**pseudo code**

We make the system quasi-incompressible by introducing a $P_{ref} =f(\frac{\rho}{\rho_{0}} = 1.02)$, where $f$ is a function. Also, note that we check that the (orthogonal) distance of a fluid particle to the wall is less than $0.9 \Delta x$. We do not want the value to be greater or equal to 1 so that the repulsive force from the wall particles would not affect particles that are not close to them (and not their neighbours). Secondly, we did not choose a value lower to ensure that the particle would be affected by the boundary and not slip through a wall unnoticed. 

## 2. Demonstration

## 3. Checking the software output

The parameters we use are the following:
<br>
max_x = (10, 7), dx = 0.5, h_fac = 1.3, t0 = 0.0, t_max = 1.5
<br>
All other parameters are default values.

To generate this simulation, we use FE scheme.
<img src="figures/crest_and_height_time.png" style="width: 700px">

We can see on these figures the location of the crest and the wave height which change with time.
<br>
We calculated the average wave velocity value using this line:
**velocities = np.mean(np.array([np.linalg.norm(p.v) for p in max_particles]))**.
<br>
The average wave velocity calculated from the location of the crest is $2.24 m/s$.
In comparison, the expected shallow water wave speed is of $ \sqrt{gh_0} = 4.90 m/s$.

We notice that the calculated average wave velocity from the simulation is less than the expected wave speed.
For this, there could be some explanations. Indeed, when the simulated wave hits a wall, its velocity decreases, in contrast to the idealised wave which travels freely. Also, the simulated wave's waveshape is obviously not ideal.


## 4. Convergence analysis

To do the convergence study, we change the resolution $\Delta{x}$ and we check the time when the wave traverse the domain and hits the wall for the first time.
Since we were running out of time, we decided to calculate the convergence of $\Delta{x}$ and time using a very approximate method. We would generate the video of the simulation with stop time approximately after the wave hits the boundary.
We would then "eyeball" the time where the wave would hit the boundary. 

Admittedly, with a larger $\Delta{x}$, estimating the time was easier to do and got progressively harder as the $\Delta{x}$ decreased since there were more particles.

<img src="figures/convergence_analysis.png" style="width: 500px">

According to a technical report by Hu, Pan, Rakhsha and Negrut, SPH method is second-order accurate. Our convergence analysis shows an order of accuracy of 1.45. Such a result could be explained by our very approximative method to get our time values at different resolutions, which prevents us to get the desired order of 2.

## 5. Artificial Pressure